 ![Spark Logo](http://archive.ics.uci.edu/ml/assets/logo.gif)
 
# Association Rule Mining Assignment

> In this assignment we will be be identifying frequent itemsets from an online retail dataset using the FP-Growth Algorithm in Pyspark.

> The data can be found on the UCI Machine Learning Repository and is described as "a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail."


#### Answer the following questions using FP-Growth


1) How many frequent itemsets were found with minimum support of .15?

In [27]:
import pandas as pd
df = pd.read_csv('Association_Rule_Mining_Data.csv', encoding = "ISO-8859-1")
df[:10]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 8:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 8:34,1.69,13047.0,United Kingdom


In [32]:
s2 =df.groupby('InvoiceNo')['StockCode'].apply(','.join)
s2 = s2.reset_index()
s2[:5].to_csv('Online Retail.txt', columns = ["StockCode"], header=False, index=False)
s2[:5] 

,InvoiceNo,StockCode
0,536365,"85123A,71053,84406B,84029G,84029E,22752,21730"
1,536366,"22633,22632"
2,536367,"84879,22745,22748,22749,22310,84969,22623,2262..."
3,536368,"22960,22913,22912,22914"
4,536369,21756


In [33]:
from pyspark.mllib.fpm import FPGrowth
from pyspark import SparkContext
sc = SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-10-1e5583a93519>:3 

In [30]:
data = sc.textFile("Online Retail.txt")
transactions = data.map(lambda line: line.strip().split(','))

In [31]:
model = FPGrowth.train(transactions, minSupport = 0.15, numPartitions = 10)
freqItemsets = model.freqItemsets().collect()
len(freqItemsets)

4241


2a) How many association rules were identified with a minimum confidence of .4?

In [42]:
freqItemsetsDict = {}

for i in range(0, len(freqItemsets)):
    freqItemsetsDict[str(freqItemsets[i].items)] = freqItemsets[i].freq
    
minConfidence = .4
rules = sorted(model._java_model.generateAssociationRules(minConfidence).collect(), 
    key=lambda x: x.confidence(), reverse=True)

len(rules)

25035


3) What is the support and confidence of the following association rule: ['21754', '22748'] => ['22745']? Go beyond reporting the values, but include an interpretation of these evaluation metrics.

In [45]:
minConfidence = .4
rules = sorted(model._java_model.generateAssociationRules(minConfidence).collect(), 
    key=lambda x: x.confidence(), reverse=True)

for j in range(0, len(rules)):
    antecedent = list(rules[j].antecedent())
    consequent = list(rules[j].consequent())
    ruleItemset = str(antecedent + consequent)
    
    try:
        support = "{0:.2f}".format(freqItemsetsDict[ruleItemset] / transactions.count())
    except KeyError:
        next
    
    confidence = "{0:.2f}".format(rules[j].confidence())
    
    try:
        print(antecedent, '=>', consequent, '(support: '+ str(support), 'confidence: '+ str(confidence) + ')')
    except NameError:
        next

['21754', '22310', '84969', '22623', '22622', '48187"'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22623', '22622', '48187"'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22623', '22622', '48187"'] => ['21755'] (support: 0.20 confidence: 1.00)
['22749'] => ['22748'] (support: 0.20 confidence: 1.00)
['22749'] => ['"84879'] (support: 0.20 confidence: 1.00)
['22749'] => ['22745'] (support: 0.20 confidence: 1.00)
['22749'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22749'] => ['21755'] (support: 0.20 confidence: 1.00)
['22749'] => ['21777'] (support: 0.20 confidence: 1.00)
['22749'] => ['22310'] (support: 0.20 confidence: 1.00)
['22749'] => ['22622'] (support: 0.20 confidence: 1.00)
['22749'] => ['22623'] (support: 0.20 confidence: 1.00)
['22749'] => ['21754'] (support: 0.20 confidence: 1.00)
['22749'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '22748'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754',

['21754', '84969', '22622', '21777', '22745'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22745'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['22310'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['21777'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['22749'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['22622'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['21754'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22622', '48187"', '22745'] =>

['22622', '"84879'] => ['22310'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['22748'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['21777'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['22749'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['22745'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['22623'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['21754'] (support: 0.20 confidence: 1.00)
['22622', '"84879'] => ['84969'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['22748'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['22310'] (support: 0.20 confidence: 1.00)
['21777', '22749', '48187"', '22745', '"84879'] => ['2262

['21754', '84969', '22623', '21777', '22748', '48187"', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22623', '21777', '22748', '48187"', '22745'] => ['22749'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22623', '21777', '22748', '48187"', '22745'] => ['22622'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '22745'] => ['22748'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '22745'] => ['22749'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '22745'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '22745'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '22745'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '22745'] => ['22622'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '22745'] => ['84969'] (support: 0.20 confidence: 1.00)
[

['22623', '22622', '21777', '22748'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22623', '22622', '21777', '22748'] => ['21755'] (support: 0.20 confidence: 1.00)
['22623', '22622', '21777', '22748'] => ['22749'] (support: 0.20 confidence: 1.00)
['22623', '22622', '21777', '22748'] => ['21754'] (support: 0.20 confidence: 1.00)
['22623', '22622', '21777', '22748'] => ['84969'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22622', '21777', '22749', '48187"', '22745', '"84879', '21755'] => ['22748'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22622', '21777', '22749', '48187"', '22745', '"84879', '21755'] => ['21754'] (support: 0.20 confidence: 1.00)
['22623', '22749', '48187"', '22745', '"84879', '21755'] => ['22310'] (support: 0.20 confidence: 1.00)
['22623', '22749', '48187"', '22745', '"84879', '21755'] => ['21777'] (support: 0.20 confidence: 1.00)
['22623', '22749', '48187"', '22745', '"84879', '21755'] => ['22748'] (support: 0.20 confidence: 1.0

...

['21754', '22310', '22622', '48187"'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22622', '48187"'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '21777', '48187"', '21755'] => ['22748'] (support: 0.20 confidence: 1.00)
['21754', '21777', '48187"', '21755'] => ['22749'] (support: 0.20 confidence: 1.00)
['21754', '21777', '48187"', '21755'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '21777', '48187"', '21755'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '21777', '48187"', '21755'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '21777', '48187"', '21755'] => ['22310'] (support: 0.20 confidence: 1.00)
['21754', '21777', '48187"', '21755'] => ['22622'] (support: 0.20 confidence: 1.00)
['21754', '21777', '48187"', '21755'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22623', '22748', '22745', '21755'] => ['22310'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22623', '22748', '22745', '21755'] => 

['21754', '84969', '22623', '22749', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22623', '22749', '22745'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22623', '22749', '22745'] => ['22622'] (support: 0.20 confidence: 1.00)
['21754', '84969', '21777', '22749', '22748', '48187"', '22745'] => ['22310'] (support: 0.20 confidence: 1.00)
['21754', '84969', '21777', '22749', '22748', '48187"', '22745'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '84969', '21777', '22749', '22748', '48187"', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '84969', '21777', '22749', '22748', '48187"', '22745'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '84969', '21777', '22749', '22748', '48187"', '22745'] => ['22622'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22749', '22748', '22745', '"84879', '21755'] => ['21777'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22749', '22748', '22745', '"84879', 

['22310', '84969', '22623', '22622', '21777', '22749', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22622', '21777', '22749', '"84879'] => ['22745'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22622', '21777', '22749', '"84879'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22622', '21777', '22749', '"84879'] => ['21754'] (support: 0.20 confidence: 1.00)
['22622', '21777', '22749', '48187"', '"84879'] => ['22310'] (support: 0.20 confidence: 1.00)
['22622', '21777', '22749', '48187"', '"84879'] => ['22748'] (support: 0.20 confidence: 1.00)
['22622', '21777', '22749', '48187"', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['22622', '21777', '22749', '48187"', '"84879'] => ['22745'] (support: 0.20 confidence: 1.00)
['22622', '21777', '22749', '48187"', '"84879'] => ['22623'] (support: 0.20 confidence: 1.00)
['22622', '21777', '22749', '48187"', '"84879'] => ['21754'] (support: 0.20 confidenc

['21754', '22310', '21777', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '"84879'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '"84879'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '"84879'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '"84879'] => ['22622'] (support: 0.20 confidence: 1.00)
['21754', '22310', '21777', '"84879'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '21777', '22749', '22748', '21755'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '21777', '22749', '22748', '21755'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '21777', '22749', '22748', '21755'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '21777', '22749', '22748', '21755'] => ['22622'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '21777', '22749', '22748'

['84969', '22623', '22622', '21777', '22748', '48187"', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['84969', '22623', '22622', '21777', '22748', '48187"', '22745'] => ['22749'] (support: 0.20 confidence: 1.00)
['22310', '22749', '22748'] => ['21777'] (support: 0.20 confidence: 1.00)
['22310', '22749', '22748'] => ['22623'] (support: 0.20 confidence: 1.00)
['22310', '22749', '22748'] => ['"84879'] (support: 0.20 confidence: 1.00)
['22310', '22749', '22748'] => ['22745'] (support: 0.20 confidence: 1.00)
['22310', '22749', '22748'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22310', '22749', '22748'] => ['21755'] (support: 0.20 confidence: 1.00)
['22310', '22749', '22748'] => ['22622'] (support: 0.20 confidence: 1.00)
['22310', '22749', '22748'] => ['84969'] (support: 0.20 confidence: 1.00)
['22310', '22749', '22748'] => ['21754'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22748', '48187"'] => ['"84879'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22748